In [ ]:
import os
import pandas as pd
import re
import openpyxl
import numpy as np
from datetime import datetime
import spreadsheet_process as sp
import unicodedata
#from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials


pd.set_option('display.max_columns', None)

In [ ]:
UF = pd.read_csv('UF.csv')
ddd = pd.read_csv('ddd.csv')
UF_dict = dict(zip(list(UF['estado']), list(UF['sigla'])))
ddd_dict = dict(zip(list(ddd['ddd']), list(ddd['UF'])))
states_by_zipcode = {
    (69900000,69999999) : 'ac',
    (57000000,57999999) : 'al',
    (69000000,69299999) : 'am',
    (69400000,69899999) : 'am',
    (68900000,68999999) : 'ap',
    (40000000,48999999) : 'ba',
    (60000000,63999999) : 'ce',
    (70000000,72799999) : 'df',
    (73000000,73699999) : 'df',
    (29000000,29999999) : 'es',
    (72800000,72999999) : 'go',
    (73700000,76799999) : 'go',
    (65000000,65999999) : 'ma',
    (30000000,39999999) : 'mg',
    (79000000,79999999) : 'ms',
    (78000000,78899999) : 'mt',
    (66000000,68899999) : 'pa',
    (58000000,58999999) : 'pb',
    (50000000,56999999) : 'pe',
    (64000000,64999999) : 'pi',
    (80000000,87999999) : 'pr',
    (20000000,28999999) : 'rj',
    (59000000,59999999) : 'rn',
    (76800000,76999999) : 'ro',
    (69300000,69399999) : 'rr',
    (90000000,99999999) : 'rs',
    (88000000,89999999) : 'sc',
    (49000000,49999999) : 'se',
    (1000000,19999999) :  'sp',
    (77000000,77999999) : 'to'
}

In [ ]:
# import pickle

# zipcode_to_file = open('st_zipcode.bin', 'wb')
# pickle.dump(states_by_zipcode, zipcode_to_file)
# zipcode_to_file.close()

In [ ]:
# st_zipcode_file = open('st_zipcode.bin', "rb")
# st_zipcode = pickle.load(st_zipcode_file)
# print(st_zipcode)

In [ ]:
def format_string(to_format):
    return ''.join(l  for l in str(to_format) if l.isalnum() or l.isspace())

def is_email_valid(email):
    if type(email) == str:
        if re.search(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", email):
            return True
        else:
            return False
    else:
        return False
    
def format_tel(tel):
    tel = str(tel)
    if len(tel) == 11 and int(tel[:2]) in ddd_dict:
        return '({}){}-{}'.format(tel[:2], tel[2:7], tel[7:])
    else:
        return tel

def state_location(zipcode: str):
    if zipcode is not np.nan:
        zip_as_numeral = int(format_string(zipcode))
        for (value1, value2) in states_by_zipcode.keys():
            if value1 <= zip_as_numeral <= value2:
                return states_by_zipcode[(value1, value2)]
    return np.nan

# def check_UF(row, UF_dict):
#     uf = str(row['Estado'])
#     tels = [row['Telefone 1'], row['Telefone 2'], row['Telefone 3']]
#     if uf in UF_dict.values():
#         return uf
#     else:
#         if uf == '' or uf == None: # search in phone number
#             for phone in tels:
#                 if phone != None and len(phone)==14 and int(phone[:2]) in ddd_dict.keys():
#                     return UF_dict[ddd_dict[int(phone[:2])]]
#             return None
#         if len(uf)>2:
#             if uf in UF_dict.keys():
#                 return UF_dict[uf]
#             else: # search in phone number
#                 for phone in tels:
#                     if phone != None and len(phone)==14 and int(phone[:2]) in ddd_dict.keys():
#                         return UF_dict[ddd_dict[int(phone[:2])]]
#                 return None


In [ ]:
if os.path.exists('./input/'):
    df = pd.read_excel('./input/listStudents6-converted.xlsx')
else:
    fpath = input()
    df = pd.read_excel(fpath)
df = df.drop(columns=['#','Último Acesso','Matrícula'])

forbidden_words = ['AAA','TESTE', 'TEST', 'TRTRIXX', 'TRIXX', 'Pedro', 'HIKARO']

# remove linhas que contenham no email ou no nome os termos acima
df = df[~df['E-Mail'].str.upper().str.contains('|'.join(forbidden_words), na=False)]
df = df[~df['Nome'].str.upper().str.contains('|'.join(forbidden_words), na=False)]
df = df[~df['QLB'].str.upper().str.contains('|'.join(forbidden_words), na=False)]

df = df.replace('^\s*$', np.nan, regex=True)

In [ ]:
''' AUTOCOMPLETE CPF  AND FORMAT'''
df['CPF'] = df['CPF'].str.replace('*', '', regex=False)
df['CPF'] = df['CPF'].str.zfill(11)

In [ ]:
colunas_padronizadas = ['Nome','QLB','Endereço', 'Curso', 'Complemento', 'Bairro', 'Cidade', 'Estado', 'CEP']
#colunas em camelcase
df[colunas_padronizadas] = df[colunas_padronizadas].apply(lambda x: x.str.title())
#remove caracteres com missing type
df[colunas_padronizadas] = df[colunas_padronizadas].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
# df['Bairro'] = df['Bairro'].apply(rm_encode_err_terms)

In [ ]:
#COLUNA: NOME
#remove linhas 
df = df[df['Nome'].notna()]
# df[df['Nome'].str.contains('454', na=False)]
#remove caracteres especiais
df['Nome'] = df['Nome'].apply(format_string)

In [ ]:
# COLUNA: QLB
df['QLB'] = df['QLB'].apply(format_string)

In [ ]:
# DATAS
df['Nascimento'] = df['Nascimento'].replace('0000-00-00', np.nan)
df['Nascimento'] = pd.to_datetime(df['Nascimento'], format='%Y-%m-%d', errors='coerce')
df['Registro'] = pd.to_datetime(df['Registro'], format='%d/%m/%Y', errors='coerce')
df['Inscrição'] = pd.to_datetime(df['Inscrição'], format='%d/%m/%Y', errors='coerce')

In [ ]:
#Coluna: SEXO
df.loc[df['CPF'] == '03791577050', 'Sexo'] = 'Masculino'
df.loc[df['CPF'] == '08025244946', 'Sexo'] = 'Feminino'
df.loc[df['CPF'] == '15241851755', 'Sexo'] = 'Masculino'
df.loc[df['CPF'] == '10419644717', 'Sexo'] = 'Feminino'

df['Sexo'] = df['Sexo'].str.lower().map({'masculino': 'M', 'feminino': 'F'})

In [ ]:
columns = ['ID', 'Número', 'CEP', 'RG', 'Telefone 1', 'Telefone 2', 'Telefone 3']
df[columns] = df[columns].replace(r'[^0-9]+', '',regex=True)

In [ ]:
''' VALIDATE CPF '''
# Show lines with invalid CPF
df[~df['CPF'].apply(sp.validate)]

In [ ]:
# Show lines with invalid Email
df[~df['E-Mail'].apply(is_email_valid)]

In [ ]:
#Coluna SEXO
#verifica Nas
df[df['Sexo'].isna()]

In [ ]:
# df['Telefone 1'] = df['Telefone 1'].apply(format_tel)
# df['Telefone 2'] = df['Telefone 2'].apply(format_tel)
# df['Telefone 3'] = df['Telefone 3'].apply(format_tel)

In [ ]:
df.loc[df['CEP'].str.len() != 8, 'CEP'] = np.nan

In [ ]:
df['Estado'] = df['Estado'].str.lower()
df[df['Estado'] == 'esparito santo'] = 'es'

In [ ]:
df['Estado'] = df['Estado'].map(lambda x: UF_dict.get(x, x))

df.loc[df['Estado'].str.contains('solteir.*|casad.*|rg|\.\.\.', na=False), 'Estado'] = np.nan
df.loc[df['Estado'].isna(), ['Estado']] = df[df['Estado'].isna()]['CEP'].apply(state_location)

In [ ]:
multiplicadores = ['cidocgs@gmail.com', 'karinekarinepereira80@gmail.com','railmalves@hotmail.com', 'serafimnago@gmail.com', 'damasogabriel@yahoo.com.br', 'jospinto25@yahoo.com.br' ]

df.loc[df['Endereço'] == 'Rua Mario Santos Braga', 'Estado'] = 'UFF'
df.loc[df['E-Mail'].isin(multiplicadores), 'Estado'] = 'UFF'

In [ ]:
df.loc[df['CPF'] == '66240379034', 'Estado'] = 'rs'
df.loc[df['CPF'] == '14646891646', 'Estado'] = 'mg'
df.loc[df['CPF'] == '14817892609', 'Estado'] = 'mg'

In [ ]:
df[df['Estado'].isna()]

In [ ]:
df['Estado'] = df['Estado'].str.upper()

In [ ]:
df['Estado'].value_counts(dropna=False)

In [ ]:
df.groupby('Estado').agg({'CPF':'nunique', 'Curso': 'count'}).sort_index()

In [ ]:
df['Curso'] = df['Curso'].str.lower()
df.drop(df.loc[df['Curso'].isin(['es', 'arte e educacao (grt)'])].index, inplace=True)

df.loc[df['Curso'] == 'educacao financeira', 'Curso'] = 'fundamentos da educacao financeira'
df['Curso'].value_counts(dropna=False).sort_index()

In [ ]:
df['Bairro'] = df['Bairro'].apply(lambda x: unicodedata.normalize('NFKD',str(x)).encode('ascii', errors='ignore').decode('utf-8'))

In [ ]:
# df.to_excel('./output/listStudents6-processed.xlsx')

In [ ]:
# import json
# # obter em: https://console.developers.google.com/
# f = open('pythonforspreadsheet-a0a4e792add1.json',) 
# creds = json.load(f)

In [ ]:
# df['Nascimento'] = df['Nascimento'].astype('string')
# df['Registro'] = df['Registro'].astype('string')
# df['Inscrição'] = df['Inscrição'].astype('string')
# df.fillna('Nan')

In [ ]:
# it maybe required give share permissions for app's service email 
# OAuthCreds should be adquired from google developers console

def Login():
    # Login into google for api's use authorization and return a api backend instance
    creds = None
    # If modifying these scopes, delete the file token.json.
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets'] # permissions
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                # OAuthCreds should be adquired from google developers console
                'OAuthCreds.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
        
    # Call the Sheets API
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()

    return sheet

def get(SHEET_API_INSTANCE, SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME):
    # get values from a sreadsheet
    result = SHEET_API_INSTANCE.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    # else:
    #     print('Data:')
    #     for row in values:
    #         print(row)
    return result

def create(SHEET_API_INSTANCE, TITLE):
    # create a sreadsheet and returns its id
    spreadsheet = {
        'properties': {
            'title': TITLE
        }
    }

    spreadsheet = SHEET_API_INSTANCE.create(body=spreadsheet,
                                        fields='spreadsheetId').execute()
    spreadsheetID = spreadsheet.get('spreadsheetId')
    print('Spreadsheet ID: {0}'.format(spreadsheetID))
    return spreadsheetID

def get_batchget(SHEET_API_INSTANCE, spreadsheet_id, range_names):
    result = SHEET_API_INSTANCE.values().batchGet(
        spreadsheetId=spreadsheet_id, ranges=range_names).execute()
    ranges = result.get('valueRanges', [])
    print('{0} ranges retrieved.'.format(len(ranges)))

    return result

def clear(SHEET_API_INSTANCE, spreadsheet_id, range):
    # The A1 notation of the values to clear.
    clear_values_request_body = {
    }

    result = SHEET_API_INSTANCE.values().clear(spreadsheetId=spreadsheet_id, range=range, body=clear_values_request_body).execute()
    return result

def update(SHEET_API_INSTANCE, spreadsheet_id, value_input_option, range_name, body):
    # update cells in a continuos range and return cells changed
    result = SHEET_API_INSTANCE.values().update(
        spreadsheetId=spreadsheet_id,
        valueInputOption=value_input_option, range=range_name, body=body).execute()
    op_info = result.get('updatedCells')
    print('{0} cells updated.'.format(op_info))
    return result

def update_batchupdate(SHEET_API_INSTANCE, spreadsheet_id, body):
    result = SHEET_API_INSTANCE.values().batchUpdate(
        spreadsheetId=spreadsheet_id, body=body).execute()
    print('{0} cells updated.'.format(result.get('totalUpdatedCells')))
    return result

In [ ]:
sheet_api_instance = Login()

In [ ]:
# The ID and range of a sample spreadsheet.
# A1 notation
# Sheet1
# Sheet1!A2:X2

# SAMPLE_SPREADSHEET_ID = '1D9Wdt-ZjHUBk6lTdwjYSBnmqn2kpIMMNKQK0aiEaRLg'
# SAMPLE_RANGE_NAME = 'Sheet1!A2:X4'
# values = get(sheet_api_instance, SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME)
# values_to_df = pd.DataFrame(values.get('values', []))
# values_to_df

In [ ]:
df['Nascimento'] = df['Nascimento'].astype('string')
df['Registro'] = df['Registro'].astype('string')
df['Inscrição'] = df['Inscrição'].astype('string')

In [ ]:
df.dtypes

In [ ]:
df = df.fillna('')

In [ ]:
SAMPLE_SPREADSHEET_ID = '1D9Wdt-ZjHUBk6lTdwjYSBnmqn2kpIMMNKQK0aiEaRLg'
range = 'Página1'
clear(sheet_api_instance, SAMPLE_SPREADSHEET_ID, range)

In [ ]:
values = df.values.tolist()
range_name = 'Página1'
body = {
    'range' : range_name,
    'values': values
}

value_input_option = 'RAW'
update_info = update(sheet_api_instance, SAMPLE_SPREADSHEET_ID, value_input_option, range_name, body)